### 1)constant==>__init__.py
### 2) utilis==>common.py
### 3) entity==>config_entity.py

In [1]:
from cnnClassifier.constants import (PARAMS_FILE_PATH, CONFIG_FILE_PATH)

In [2]:
print(CONFIG_FILE_PATH)

D:\project\entbappy(Bappy Ahmed)\End-to-End-Chest-Cancer-Classification-using-MLflow-DVC\config\config.yaml


In [3]:
import box
dir(box)

['Box',
 'BoxError',
 'BoxKeyError',
 'BoxList',
 'ConfigBox',
 'DDBox',
 'SBox',
 '__all__',
 '__author__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'box',
 'box_from_file',
 'box_from_string',
 'box_list',
 'config_box',
 'converters',
 'exceptions',
 'from_file',
 'shorthand_box']

In [4]:
import yaml
from box import ConfigBox
from box.exceptions import BoxValueError
from ensure import ensure_annotations
from pathlib import Path
import os
@ensure_annotations
def read_yaml(file_path: Path)-> ConfigBox:
    try:
        with open(file_path) as yaml_file:
            content = yaml.safe_load(yaml_file)
        return ConfigBox(content)
    except BoxValueError:
        raise ValueError("yaml file is empty")
    except Exception as e:
        raise e


@ensure_annotations
def create_directories(path_to_directories:list, verbose=True):
    for path in path_to_directories:
        os.makedirs(path, exist_ok=True)
    if verbose:
        logger.info(f"the file created at: {path}")

@ensure_annotations
def get_size(path: Path) -> str:
    """get size in KB

    Args:
        path (Path): path of the file

    Returns:
        str: size in KB
    """
    size_in_kb = round(os.path.getsize(path)/1024)
    return f"~ {size_in_kb} KB"


### importance of dataclass decorator?
#### dataclass is used for storing data in class and attribute value cann't change with frozen=True

In [5]:
import dataclasses

In [6]:
from dataclasses import dataclass

@dataclass(frozen=True) #forzen=True it is a immutalbe, we can't cahange attribute values
class DataIngestionConfigEntity:
    root_dir: Path
    source_URL: Path
    local_data_file: Path
    unzip_dir: Path

### 4) config in src file 


In [7]:
import gdown
class ConfigrationManager:
    def __init__(self, config_file_path = CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingenstion_config(self) -> DataIngestionConfigEntity:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_entity = DataIngestionConfigEntity(root_dir=config.root_dir, source_URL=config.source_URL, local_data_file=config.local_data_file, unzip_dir=config.unzip_dir)

        return data_entity

### 5) components

In [8]:
#download the data from source_url
import gdown 
import zipfile

class DataIngestion:
    def __init__(self, config: DataIngestionConfigEntity):
        self.config = config

    def download_file(self)->str:
        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

### 6) pipeline

In [9]:
from cnnClassifier import logger
STAGE_NAME = "Data Ingestion stage"

class DataIngestionTrainingPipeline:
    def __init__(self):
        pass

    def main(self):
        config = ConfigrationManager()
        data_ingestion_config = config.get_data_ingenstion_config()
        data_ingestion = DataIngestion(config=data_ingestion_config)
        data_ingestion.download_file()
        data_ingestion.extract_zip_file()




if __name__ == '__main__':
    try:
        logger.info(f">>>>>> stage {STAGE_NAME} started <<<<<<")
        obj = DataIngestionTrainingPipeline()
        obj.main()
        logger.info(f">>>>>> stage {STAGE_NAME} completed <<<<<<\n\nx==========x")
    except Exception as e:
        logger.exception(e)
        raise e

[2024-07-19 16:12:47,222: INFO: 331079319: >>>>>> stage Data Ingestion stage started <<<<<<]
[2024-07-19 16:12:47,236: INFO: 1612917504: the file created at: artifacts]
[2024-07-19 16:12:47,240: INFO: 1612917504: the file created at: artifacts/data_ingestion]
[2024-07-19 16:12:47,243: INFO: 664224474: Downloading data from https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1z0mreUtRmR-P-magILsDR3T7M6IkGXtY
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1z0mreUtRmR-P-magILsDR3T7M6IkGXtY&confirm=t&uuid=4c4e205d-0d80-4a64-9a37-d5bb450d3beb
To: D:\project\entbappy(Bappy Ahmed)\End-to-End-Chest-Cancer-Classification-using-MLflow-DVC\research\artifacts\data_ingestion\data.zip
100%|██████████| 49.0M/49.0M [00:02<00:00, 22.7MB/s]

[2024-07-19 16:12:54,418: INFO: 664224474: Downloaded data from https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing into file artifacts/data_ingestion/data.zip]


[2024-07-19 16:12:55,559: INFO: 331079319: >>>>>> stage Data Ingestion stage completed <<<<<<

x==========x]
